Coding Exercise

- Data source: free API access from Coinapi.io
- API documentation: https://docs.coinapi.io/

### Set up

In [178]:
import requests
import pandas as pd
import datetime as dt
import pytz

In [158]:
# limit range from 100 to 10000
limit = 1000

# free api key can be requested at: https://www.coinapi.io/pricing?apikey
api_key = '2862B3F7-A7BA-4E50-A006-6CE261524C79' 

url = 'https://rest.coinapi.io/v1/trades/COINBASE_SPOT_BTC_USD/latest?limit=' + str(limit)
headers = {'X-CoinAPI-Key' : api_key}

In [307]:
response = requests.get(url, headers=headers)
trades = pd.DataFrame(response.json())

In [308]:
trades.head()

,price,size,symbol_id,taker_side,time_coinapi,time_exchange,uuid
0,7365.47,0.613260,COINBASE_SPOT_BTC_USD,BUY,2018-07-18T00:43:07.7633715Z,2018-07-18T00:43:07.7340000Z,6c5c1af7-3360-483f-9253-3c6b85ef2ec0
1,7365.47,0.003402,COINBASE_SPOT_BTC_USD,BUY,2018-07-18T00:43:07.7633715Z,2018-07-18T00:43:07.7340000Z,8621cf75-8c73-4c6e-b64a-3d519afd0f66
2,7365.47,0.080727,COINBASE_SPOT_BTC_USD,BUY,2018-07-18T00:43:07.7633715Z,2018-07-18T00:43:07.7340000Z,66d06815-2a1c-47b8-8a67-da92a24123a3
3,7365.46,0.008595,COINBASE_SPOT_BTC_USD,SELL,2018-07-18T00:43:04.4821231Z,2018-07-18T00:43:04.4730000Z,e3418735-e461-47bf-9be1-b5a09b864624
4,7365.46,0.543200,COINBASE_SPOT_BTC_USD,SELL,2018-07-18T00:43:03.4821356Z,2018-07-18T00:43:03.4630000Z,3e91464a-7cd4-40be-af2a-81c46d5147a0


There is some delay in the live data, so we will use the data between 6 minutes ago and 7 minutes ago.

In [309]:
# Get current UTC time
now = dt.datetime.utcnow()
# Get current minute
current_minute = dt.datetime(now.year,now.month,now.day,now.hour,now.minute)
# Get 1 and 2 mins ago
one_min_ago = current_minute - dt.timedelta(minutes=6)
two_min_ago = current_minute - dt.timedelta(minutes=7)

In [310]:
# Convert string to datetime object
time_exchange = trades['time_exchange'].apply(lambda x: dt.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%f0Z" ))
# Select rows between 1 min ago and 2 min ago
mask = (time_exchange < one_min_ago) & (time_exchange > two_min_ago)
trade_last_min = trades[mask]

In [311]:
trade_last_min

,price,size,symbol_id,taker_side,time_coinapi,time_exchange,uuid
6,7365.47,0.012990,COINBASE_SPOT_BTC_USD,BUY,2018-07-18T00:42:59.8571328Z,2018-07-18T00:42:59.8570000Z,e32b8d77-f633-4431-99ba-5940b8066e79
7,7365.32,0.021497,COINBASE_SPOT_BTC_USD,SELL,2018-07-18T00:42:57.5290114Z,2018-07-18T00:42:57.5200000Z,5e0dd8cc-453c-4ace-98e4-f9c7ee13a561
8,7365.58,0.036226,COINBASE_SPOT_BTC_USD,BUY,2018-07-18T00:42:53.7477714Z,2018-07-18T00:42:53.7340000Z,96611a72-df3d-4b01-a7c6-68f5447dac97
9,7365.57,0.001154,COINBASE_SPOT_BTC_USD,BUY,2018-07-18T00:42:53.7477714Z,2018-07-18T00:42:53.7340000Z,e89685eb-fe46-4f01-9c10-7236c9891524
10,7365.58,0.065127,COINBASE_SPOT_BTC_USD,BUY,2018-07-18T00:42:52.5290107Z,2018-07-18T00:42:52.3370000Z,d5674c43-735b-4e1f-b38d-c5b397e42519
11,7365.58,0.001425,COINBASE_SPOT_BTC_USD,BUY,2018-07-18T00:42:52.5290107Z,2018-07-18T00:42:52.3370000Z,64ef97d8-745c-4f50-8eae-b014a1ad035c
12,7365.52,0.019200,COINBASE_SPOT_BTC_USD,SELL,2018-07-18T00:42:50.7321562Z,2018-07-18T00:42:50.6800000Z,6ab25b0a-d5f2-415e-9b7d-9bf164e02c66
13,7365.62,0.541756,COINBASE_SPOT_BTC_USD,BUY,2018-07-18T00:42:49.3884095Z,2018-07-18T00:42:49.2380000Z,bde531c4-5dc8-49ab-a59d-b3f47709319b
14,7365.62,0.001425,COINBASE_SPOT_BTC_USD,BUY,2018-07-18T00:42:49.3884095Z,2018-07-18T00:42:49.2380000Z,d5494dbc-d9a0-4784-8b57-ad222276018b
15,7365.37,0.130516,COINBASE_SPOT_BTC_USD,SELL,2018-07-18T00:42:42.9665394Z,2018-07-18T00:42:42.9410000Z,c2790eae-fe3a-407b-92be-35a5f82347d1
